In [63]:
# import libraries
import pandas as pd
import glob

In [64]:
# use list comprehention and glob to create one db from many
# works because he each db has the same columns
df = pd.concat([pd.read_csv(x) for x in glob.glob('data/Kickstarter*.csv')], ignore_index = True)

In [65]:
df.head(2)

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,61,Support great art! Join us as we re-stage the ...,"{""id"":254,""name"":""Performances"",""slug"":""dance/...",4618,US,the United States,1579292017,"{""id"":2092817311,""name"":""Brooklyn Ballet"",""slu...",USD,$,...,revisionist-history-2,https://www.kickstarter.com/discover/categorie...,True,True,successful,1583025192,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",4618.0,domestic
1,52,JinBucha is a new kind of Brewery in North Par...,"{""id"":307,""name"":""Drinks"",""slug"":""food/drinks""...",3461,US,the United States,1446051515,"{""id"":1468694331,""name"":""Jing Chen"",""slug"":""ji...",USD,$,...,jinbucha-a-modern-kombucha-tasting-room-in-nor...,https://www.kickstarter.com/discover/categorie...,False,True,failed,1450118057,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3461.0,domestic


In [66]:
df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'country_displayable_name', 'created_at', 'creator',
       'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'launched_at', 'location', 'name', 'permissions', 'photo', 'pledged',
       'profile', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219688 entries, 0 to 219687
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             219688 non-null  int64  
 1   blurb                     219680 non-null  object 
 2   category                  219688 non-null  object 
 3   converted_pledged_amount  219688 non-null  int64  
 4   country                   219688 non-null  object 
 5   country_displayable_name  219688 non-null  object 
 6   created_at                219688 non-null  int64  
 7   creator                   219688 non-null  object 
 8   currency                  219688 non-null  object 
 9   currency_symbol           219688 non-null  object 
 10  currency_trailing_code    219688 non-null  bool   
 11  current_currency          219688 non-null  object 
 12  deadline                  219688 non-null  int64  
 13  disable_communication     219688 non-null  b

In [68]:
# check for duplicates of individual projects
dup_id = len(df[df.duplicated(subset='id')])
print(f"There are {dup_id} ids listed more than once.")

There are 27669 listed more than once.


In [69]:
df_dup_rows = df[df.duplicated(subset='id', keep=False)]
df_dup_rows.shape

(55336, 38)

In [70]:
# discover how many rows contain duplicates
dup_rows = len(df_dup_rows)
print(f"There are {dup_rows} rows containing duplicates.")
print(f"That is an average of {(dup_rows/dup_id):.2f} copies per duplicated item.")

There are 55336 rows containing duplicates.
That is an average of 2.00 copies per duplicated item.


In [71]:
# check state values
# combine canceled with failed?  & drop live?
# make booklean
print(df['state'].value_counts())
print("---"*15)

successful    128156
failed         76137
canceled        9138
live            6257
Name: state, dtype: int64
---------------------------------------------


In [72]:
# verify that 'is_starrable' largely associated with live campaigns
print(df['is_starrable'].value_counts())
print("---"*15)
print('')

live_starrable = len(df.loc[(df['state'].isin(['live'])) & (df['is_starrable'])])

print(f'There are {live_starrable} rows are both classified as "live" and "is_starrable".')
print('Therefore a campaign must be live to be strarrable.')

False    213465
True       6223
Name: is_starrable, dtype: int64
---------------------------------------------

successful    128156
failed         76137
canceled        9138
live            6257
Name: state, dtype: int64
---------------------------------------------
There are 6223 rows are both classified as "live" and "is_starrable".
Therefore a campaign must be live to be strarrable.


In [73]:
# future data columns, includes 4 mostly null columns(17)
'''
'spotlight' only applies to successfully funded projects,
'is_starrable' is only true with live campaigns,
'''

future_data = ['backers_count', 'converted_pledged_amount','created_at', 'friends',
                'is_backing','is_starrable', 'is_starred', 'permissions' 'photo',
                'pledged','profile','source_url', 'spotlight', 'staff_pick',
                'state_changed_at', 'urls', 'usd_pledged']

In [74]:
# columns to retain as-is (3)
keepers = ['country', 'goal', 'id']

#category column to be cleaned (1)

# duplicate project descriptions - which, if any, to keep? (3)
describers = ['blurb', 'name', 'slug']

# time columns, to determine length of campaign (2)
# as per Kickstart this is 1 - 60 days
time = ['launched_at', 'deadline']


In [75]:
# extraneous columns (5)

print(df['disable_communication'].value_counts())
print("---"*15)
print('')

# check for duplicate creators
print(f"There are {len(df[df.duplicated(subset='creator')])} creators listed more than once.")

'''
'disable communications' only contains the value 'false',
'fx_rate' is the foriegn exchange rate,
'country_displayable_name' duplicate information,
'creator' is high cardinality
'location' is high cardinatlity (16910)

'''
extra_columns = ['country_displayable_name','creator','disable_communication','fx_rate', 'location']


False    219688
Name: disable_communication, dtype: int64
---------------------------------------------

There are 669 creators listed more than once.


In [76]:
# columns to evaluate - currency (6)
to_evaluate = ['currency', 'currency_symbol', 'currency_trailing_code',
                'current_currency', 'static_usd_rate', 'usd_type']

In [77]:
# check for number of unique 'category' items
# I suggest splitting 'category' into component columns - SW
df['category'].nunique()

171

In [78]:
# split category into several columns
# must include map(eval)
df2 = df['category'].map(eval).apply(pd.Series)
df2.head()

,id,name,slug,position,parent_id,parent_name,color,urls
0,254,Performances,dance/performances,1,6.0,Dance,10917369,{'web': {'discover': 'http://www.kickstarter.c...
1,307,Drinks,food/drinks,4,10.0,Food,16725570,{'web': {'discover': 'http://www.kickstarter.c...
2,307,Drinks,food/drinks,4,10.0,Food,16725570,{'web': {'discover': 'http://www.kickstarter.c...
3,311,Food Trucks,food/food trucks,8,10.0,Food,16725570,{'web': {'discover': 'http://www.kickstarter.c...
4,28,Product Design,design/product design,5,7.0,Design,2577151,{'web': {'discover': 'http://www.kickstarter.c...


In [79]:
df1=df.copy()

In [80]:
#Attempt to drop category column and add splits in one step - failed
# 
# pd.concat([df1.drop(['category'], axis=1), (df1['category'].map(eval).apply(pd.Series))], axis=1)
# df1.head()


In [81]:
#drop extraneous category columns
df2=df2.drop(['id', 'position', 'parent_id', 'color', 'urls'], axis=1)
df2.head()

,name,slug,parent_name
0,Performances,dance/performances,Dance
1,Drinks,food/drinks,Food
2,Drinks,food/drinks,Food
3,Food Trucks,food/food trucks,Food
4,Product Design,design/product design,Design


In [82]:
# the main 15 categories are listed in 'parent_name'
# 'name' contains the slug categories, useful for creating front-end drop downs
# perhaps the earliest projects only had 'name'?
# the 'slug' column pulled from 'category' includes both pieces of info
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219688 entries, 0 to 219687
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   name         219688 non-null  object
 1   slug         219688 non-null  object
 2   parent_name  211388 non-null  object
dtypes: object(3)
memory usage: 5.0+ MB


In [91]:
# these lists will be useful for frontend,
print('Main Categories:')
print(df2['parent_name'].value_counts().sort_index())
print(f'Sub Categories:', df2['name'].nunique())
print(f'Main/Sub Combos:', df2['slug'].nunique())

Main Categories:
Art             21017
Comics           8939
Crafts           6805
Dance            2804
Design           8998
Fashion         12787
Film & Video    28524
Food            16261
Games           14269
Journalism       5452
Music           27388
Photography      8279
Publishing      21242
Technology      21786
Theater          6837
Name: parent_name, dtype: int64
Sub Categories: 161
Main/Sub Combos: 171


In [103]:
# this lists will be useful for frontend
# how to format this for them?
df2['slug'].value_counts().sort_index()


art                    985
art/ceramics           487
art/conceptual art    1270
art/digital art       2443
art/illustration      3190
                      ... 
theater/festivals      926
theater/immersive      603
theater/musical       1521
theater/plays         2404
theater/spaces         333
Name: slug, Length: 171, dtype: int64